In [5]:
# Necessary Library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector as sql

In [6]:
# Load the Data Set Bird_Monitoring_Data_FOREST.XLSX

data_forest = pd.ExcelFile('Bird_Monitoring_Data_FOREST.XLSX')

df_forest = [] # Empty list

# Process of merge the All Sheets in the Data
for sheet in data_forest.sheet_names:
    df = pd.read_excel(data_forest, sheet_name=sheet)
    df_forest.append(df)
    
merged_df = pd.concat(df_forest , ignore_index=True)

# Convert the Data into Excel
merged_df.to_excel('Birds_forest.xlsx', index=False)

In [7]:
# Load the Data Set Bird_Monitoring_Data_GRASSLAND.XLSX


data_grassland = pd.ExcelFile('Bird_Monitoring_Data_GRASSLAND.XLSX')

df_grassland = []

# Process Of the Merge the All Sheets in the Data ( Optimized Code )
for sheet in data_grassland.sheet_names:
    df = pd.read_excel(data_grassland, sheet_name=sheet)
    df_grassland.append(df)
    
merged_dfg = pd.concat(df_grassland , ignore_index=False)

merged_dfg.to_excel('Birds_Grassland.xlsx',index=False)

C:\Users\Vijay K\AppData\Local\Temp\ipykernel_18880\1059724042.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_dfg = pd.concat(df_grassland , ignore_index=False)


In [8]:
# Check the Unique Columns
sets = set(merged_df.columns).intersection(set(merged_dfg.columns))

# 
a =merged_df[['Sky', 'Scientific_Name', 'ID_Method', 'Common_Name', 'Wind', 'AOU_Code', 'Sub_Unit_Code', 'AcceptedTSN', 'Initial_Three_Min_Cnt', 'End_Time', 'Admin_Unit_Code', 'Start_Time', 'Disturbance', 'Interval_Length', 'Plot_Name', 'Flyover_Observed', 'Sex', 'Location_Type', 'Regional_Stewardship_Status', 'Temperature', 'Observer', 'PIF_Watchlist_Status', 'Visit', 'Humidity', 'Year', 'Date', 'Distance']]
b = merged_dfg[['Sky', 'Scientific_Name', 'ID_Method', 'Common_Name', 'Wind', 'AOU_Code', 'Sub_Unit_Code', 'AcceptedTSN', 'Initial_Three_Min_Cnt', 'End_Time', 'Admin_Unit_Code', 'Start_Time', 'Disturbance', 'Interval_Length', 'Plot_Name', 'Flyover_Observed', 'Sex', 'Location_Type', 'Regional_Stewardship_Status', 'Temperature', 'Observer', 'PIF_Watchlist_Status', 'Visit', 'Humidity', 'Year', 'Date', 'Distance']]

concat_Data = pd.concat([a , b])

concat_Data.to_csv("Merged Data.csv",index=False)


# Preprocessing

In [9]:
# PreProcessing
forest = pd.read_csv("Merged Data.csv")

# Find the Missing Value
forest.isnull().sum() # (Sub_Unit_Code-16355 , ID_Method -2 ,Distance-92, Sex-5183 ,AcceptedTSN-33 )

# Sub_Unit_Code-7824 Missing Value So I Drop the Subunit
forest = forest.drop(columns='Sub_Unit_Code')

# Fill the missing Value
forest['Sex'] = forest['Sex'].fillna('Unknown')

# Fill the Missing Value
forest['ID_Method'] = forest['ID_Method'].ffill()

# Missing Value in Distance Column is Fill with Zero
forest['Distance'] = forest['Distance'].fillna(0)

# Setting the Format To Date
forest['Date'] = pd.to_datetime(forest['Date'],errors='coerce')

# AcceptedTSN Fill with mean Value
forest['AcceptedTSN'] = forest['AcceptedTSN'].fillna(forest['AcceptedTSN'].mean)


# Convert the Format Object to Time (hour)
forest['Start_Time'] = pd.to_datetime(forest['Start_Time']).dt.hour
forest['End_Time'] = pd.to_datetime(forest['End_Time']).dt.hour


C:\Users\Vijay K\AppData\Local\Temp\ipykernel_18880\1819894215.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  forest['Start_Time'] = pd.to_datetime(forest['Start_Time']).dt.hour
C:\Users\Vijay K\AppData\Local\Temp\ipykernel_18880\1819894215.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  forest['End_Time'] = pd.to_datetime(forest['End_Time']).dt.hour


# Analysis Of Birds - Forest & Grassland


In [10]:
# ●	Seasonal Trends: Analyze the Date and Year columns to detect 
#   patterns in bird sightings across different seasons or years.

forest['Year'] = forest['Date'].dt.year
forest['Month'] = forest['Date'].dt.month


forest['Season'] =forest["Month"].map({
    12: 'Winter', 1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Autumn', 10: 'Autumn', 11: 'Autumn'
})
forest.groupby(['Year','Season']).size().reset_index(name='Birds_count').sort_values(by='Birds_count',ascending=False).head(5)


,Year,Season,Birds_count
1,2018,Summer,11481
0,2018,Spring,5596


In [11]:
# ●	Observation Time: Study the Start_Time and End_Time to determine if
#   specific time windows correlate with higher bird activity.

bins = [5 , 7 , 9 , 11 , 13 , 15]
labels = ['5 AM to 7 AM','7AM to 9AM','9AM to 11AM','11AM to 1PM','1PM to 3PM']

forest['Time Window'] = pd.cut(forest['Start_Time'] , bins=bins,labels=labels,right=False)

forest.groupby('Time Window').size().reset_index(name="Birds Count")

C:\Users\Vijay K\AppData\Local\Temp\ipykernel_18880\4016703599.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  forest.groupby('Time Window').size().reset_index(name="Birds Count")


,Time Window,Birds Count
0,5 AM to 7 AM,5534
1,7AM to 9AM,8110
2,9AM to 11AM,3433
3,11AM to 1PM,0
4,1PM to 3PM,0


In [12]:
# ●	Location Insights: Group data by Location_Type 
#   (e.g., Grassland) to identify biodiversity hotspots.
df = forest
pd.DataFrame(df.columns)

ft = df.loc[(df['Location_Type'] == 'Grassland')]

ft = df.groupby(['Location_Type','Scientific_Name']).size().reset_index(name='hotspots')
ft.loc[(ft['Location_Type'] == 'Grassland')].sort_values(by='hotspots',ascending=False).head(10)

,Location_Type,Scientific_Name,hotspots
120,Grassland,Cardinalis cardinalis,565
201,Grassland,Sturnus vulgaris,516
197,Grassland,Spizella pusilla,506
169,Grassland,Passerina cyanea,485
111,Grassland,Ammodramus savannarum,382
109,Grassland,Agelaius phoeniceus,379
196,Grassland,Spizella passerina,368
203,Grassland,Thryothorus ludovicianus,365
121,Grassland,Carduelis tristis,362
116,Grassland,Bombycilla cedrorum,302


In [13]:
# ●	Plot-Level Analysis: Compare observations across 
#   different Plot_Name to see which plots 
#   attract more species or specific kinds of birds...


df.groupby('Plot_Name').size().reset_index(name='Count').sort_values(by='Count',ascending=False).head(10)


,Plot_Name,Count
381,MONO-0054,204
415,MONO-0094,118
296,MANA-0076,116
423,MONO-0102,88
377,MONO-0050,77
372,MONO-0045,73
88,ANTI-0188,72
51,ANTI-0124,71
407,MONO-0083,68
306,MANA-0120,63


In [14]:
# ●	Diversity Metrics: Count unique species (Scientific_Name) observed and their distribution across Location_Type.


df.groupby('Location_Type')['Scientific_Name'].nunique().reset_index(name='Unique_Species_Count').sort_values(by='Unique_Species_Count')
# df.pivot_table(index='Scientific_Name',values='Scientific_Name',aggfunc='count')

,Location_Type,Unique_Species_Count
1,Grassland,107
0,Forest,108


In [15]:
# ●	Activity Patterns: Check the Interval_Length and ID_Method 
#   columns to identify the most 
#   common activity types (e.g., Singing).

df[['ID_Method','Interval_Length']]

sing = df.groupby(['ID_Method','Interval_Length']).size().reset_index(name='Most common activity types')
sing.loc[sing['ID_Method'] == 'Singing'].sort_values(by='Most common activity types',ascending=False )

,ID_Method,Interval_Length,Most common activity types
4,Singing,0-2.5 min,5689
5,Singing,2.5 - 5 min,1828
6,Singing,5 - 7.5 min,1263
7,Singing,7.5 - 10 min,1085


In [27]:
'''●Sex Ratio: 
Analyze the Sex column to understand the male-to-female 
ratio for different species.'''

# df.Sex.unique()

df.Common_Name.nunique()

df.groupby(['Common_Name' ,'Sex']).size().reset_index(name='Gender Count').sort_values(by='Gender Count',ascending=False)

,Common_Name,Sex,Gender Count
204,Red-eyed Vireo,Undetermined,680
70,Carolina Wren,Unknown,646
168,Northern Cardinal,Undetermined,546
169,Northern Cardinal,Unknown,525
115,European Starling,Undetermined,516
...,...,...,...
218,Savannah Sparrow,Male,1
206,Red-headed Woodpecker,Unknown,1
74,Chestnut-sided Warbler,Male,1
230,Tennessee Warbler,Undetermined,1


In [17]:
# ●	Weather Correlation: Explore how Temperature, Humidity, Sky, and Wind 
#   impact observations, such as the number of birds or their distances.

# Check correlation
df[['Temperature','Humidity','Initial_Three_Min_Cnt']].corr()


,Temperature,Humidity,Initial_Three_Min_Cnt
Temperature,1.000000,-0.353349,-0.028115
Humidity,-0.353349,1.000000,0.043873
Initial_Three_Min_Cnt,-0.028115,0.043873,1.000000


In [18]:
# Group by Wind Distance
forest.groupby(['Sky','Wind','Distance'])['Initial_Three_Min_Cnt'].sum().reset_index().sort_values(by='Initial_Three_Min_Cnt',ascending=False).head(10)

,Sky,Wind,Distance,Initial_Three_Min_Cnt
52,Partly Cloudy,Light air movement (1-3 mph) smoke drifts,50 - 100 Meters,938
53,Partly Cloudy,Light air movement (1-3 mph) smoke drifts,<= 50 Meters,868
7,Clear or Few Clouds,Light air movement (1-3 mph) smoke drifts,50 - 100 Meters,762
8,Clear or Few Clouds,Light air movement (1-3 mph) smoke drifts,<= 50 Meters,756
2,Clear or Few Clouds,Calm (< 1 mph) smoke rises vertically,<= 50 Meters,502
55,Partly Cloudy,Light breeze (4-7 mph) wind felt on face,50 - 100 Meters,458
1,Clear or Few Clouds,Calm (< 1 mph) smoke rises vertically,50 - 100 Meters,428
47,Partly Cloudy,Calm (< 1 mph) smoke rises vertically,<= 50 Meters,410
46,Partly Cloudy,Calm (< 1 mph) smoke rises vertically,50 - 100 Meters,381
19,Cloudy/Overcast,Light air movement (1-3 mph) smoke drifts,50 - 100 Meters,376


In [19]:
disturbance = df.groupby('Disturbance').size().reset_index(name='Count Of effect').sort_values(by="Count Of effect",ascending=False)
disturbance

,Disturbance,Count Of effect
1,No effect on count,8029
3,Slight effect on count,6750
0,Moderate effect on count,1819
2,Serious effect on count,479


In [ ]:
df.loc[0]

df.groupby(['Distance','Observer']).size().reset_index(name='Count_Observer')

,Distance,Observer,Count_Observer
0,0,Brian Swimelar,544
1,0,Elizabeth Oswald,519
2,0,Kimberly Serno,423
3,50 - 100 Meters,Brian Swimelar,2050
4,50 - 100 Meters,Elizabeth Oswald,3679
5,50 - 100 Meters,Kimberly Serno,2510
6,<= 50 Meters,Brian Swimelar,2190
7,<= 50 Meters,Elizabeth Oswald,2193
8,<= 50 Meters,Kimberly Serno,2969


In [21]:
# ●	Flyover Frequency: Examine the Flyover_Observed column to detect trends in bird behavior during observation.

df.loc[0]

df.groupby(['Observer','Flyover_Observed']).size().reset_index(name='Count od Trend')


,Observer,Flyover_Observed,Count od Trend
0,Brian Swimelar,False,4240
1,Brian Swimelar,True,544
2,Elizabeth Oswald,False,5872
3,Elizabeth Oswald,True,519
4,Kimberly Serno,False,5479
5,Kimberly Serno,True,423


In [35]:
#  ●Observer Bias: Analyze data by Observer to check if specific 
#   individuals report more observations or certain species.

df.groupby(['Observer','Common_Name'])['Scientific_Name'].count().reset_index().sort_values(by='Scientific_Name',ascending=False).head(10)
# df.loc[0]

,Observer,Common_Name,Scientific_Name
53,Brian Swimelar,Northern Cardinal,407
223,Kimberly Serno,Carolina Wren,406
149,Elizabeth Oswald,Northern Cardinal,384
111,Elizabeth Oswald,Carolina Wren,383
259,Kimberly Serno,Northern Cardinal,369
242,Kimberly Serno,European Starling,306
164,Elizabeth Oswald,Red-eyed Vireo,299
272,Kimberly Serno,Red-eyed Vireo,299
240,Kimberly Serno,Eastern Tufted Titmouse,276
112,Elizabeth Oswald,Cedar Waxwing,237


In [33]:
# ●	Visit Patterns: Evaluate the Visit column to see how repeated visits affect
#   species count or diversity.

df.groupby('Visit')['Sky'].count().reset_index(name='Count of Visit')   

df.groupby('Visit')["Scientific_Name"].nunique().reset_index(name = 'UNique_Species_Count')

visit_summary = df.groupby('Visit').agg({
    "Scientific_Name" : pd.Series.nunique,
    'Initial_Three_Min_Cnt' : 'sum'}).reset_index()

visit_summary.columns = ['Visit','Unique_Birds_Count','Total_Birds']
visit_summary.to_csv('Visit.csv',index=False)
visit_summary

,Visit,Unique_Birds_Count,Total_Birds
0,1,119,3957
1,2,100,3733
2,3,74,1452


In [36]:
# ●	Watchlist Trends: Use the PIF_Watchlist_Status and 
# Regional_Stewardship_Status to identify trends in species 
# that are at risk or require conservation focus.

watchlist_counts = df[df['PIF_Watchlist_Status'] == True].groupby('Scientific_Name').size().reset_index(name='Watchlist_Count')
stewardship_counts = df[df['Regional_Stewardship_Status'] == True].groupby('Scientific_Name').size().reset_index(name='Regional_Stewardship_Count')

# Combine both
combined_status = pd.merge(watchlist_counts, stewardship_counts, on='Scientific_Name', how='outer').fillna(0)
combined_status.sort_values(by='Watchlist_Count', ascending=False)

,Scientific_Name,Watchlist_Count,Regional_Stewardship_Count
7,Hylocichla mustelina,309.0,309.0
6,Helmitheros vermivorus,31.0,31.0
18,Setophaga discolor,25.0,25.0
16,Setophaga cerulea,7.0,0.0
3,Empidonax traillii,2.0,0.0
9,Oporornis formosus,2.0,2.0
23,Vermivora cyanoptera,1.0,0.0
8,Melanerpes erythrocephalus,1.0,0.0
0,Ceryle alcyon,0.0,4.0
5,Guiraca caerulea,0.0,15.0


In [37]:
df.groupby(['AOU_Code','Scientific_Name']).size().reset_index(name="Count").sort_values(by="Count",ascending=False).head(10)
df.pivot_table(index='AOU_Code',values='Scientific_Name', aggfunc='count').sort_values(by='Scientific_Name',ascending=False).reset_index(names=['AOU_Code','Observational Count'])

,AOU_Code,Scientific_Name
0,NOCA,1160
1,CARW,1011
2,REVI,739
3,ETTI,728
4,INBU,635
...,...,...
121,NAWA,1
122,UNWA,1
123,TEWA,1
124,SASP,1
